### Importing modules

In [4]:
# Importing modules to download and process the data
import pandas_datareader as web
import random
from datetime import datetime
from pprint import pprint
import yfinance as yf
import pandas as pd

# Importing modules to plot (using Bokeh)
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.palettes import Spectral11, colorblind, Inferno, BuGn, brewer
from bokeh.models import HoverTool, value, LabelSet, Legend, ColumnDataSource,LinearColorMapper,BasicTicker, PrintfTickFormatter, ColorBar

# Ignoring the warnings
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

### Plotting - NASDAQ indices for each sector

In [ ]:
# NASDAQ
indices = ['XLE','XLF', 'XLP','XLB','XLK', 'XLI', 'XLU', 'TISHX', 'XLY', 'TRREX', 'XLV']
newcols = ['Energy', 'Financials', 'Cons_Defensive', 'Materials', 'Technology', 'Industrials', 'Utilities', 'Communication', 'Cons_Cyclical', 'Real_Estate', 'Healthcare']

df = web.DataReader(indices, data_source='yahoo', start='1998-01-01', end='2020-03-28')['Close'] # Downloading data
df.columns=newcols # Replacing the column headers to more readable (names of sectors)

df = df.dropna() # Dropping rows with null values to have concurrent time series among the stocks
df = df.rolling(window=30).mean() # Rolling average of the time series data is taken to get smooth curves

# Normalising the data to scale between 0 and 1
for col in df.columns:
    df[col] = (df[col]-df[col].min())/(df[col].max()-df[col].min())

# Defining tools, canvas and parameters for the plot
TOOLS = 'crosshair,save,pan,box_select,box_zoom,wheel_zoom,hover,reset'
colors = [(random.randrange(0, 255), random.randrange(0, 255),  random.randrange(0, 255)) for i in range(len(df.columns))]
plot = figure(plot_width=1600, plot_height=400, title="Title", y_axis_type="linear", x_axis_type='datetime', tools=TOOLS)
plot.legend.location = "top_left"
plot.xaxis.axis_label = 'Year'
plot.yaxis.axis_label = 'Normalised Stock'

# Plotting each of the time series.
for i in range(len(df.columns)):
    plot.line(x=df.index, y=df[df.columns[i]], legend_label= df.columns[i], line_color=colors[i], line_width = 3)                               

# Showing the output plots
output_file("Stocks.html", title="Stocks")
show(plot)

### 2008 Crash - NASDAQ

#### Creating a dataframe with the data of various sector indices

In [14]:
indices = ['XLE','XLF', 'XLP','XLB','XLK', 'XLI', 'XLU', 'TISHX', 'XLY', 'TRREX', 'XLV']
newcols = ['Energy', 'Financials', 'Cons_Defensive', 'Materials', 'Technology', 'Industrials', 'Utilities', 'Communication', 'Cons_Cyclical', 'Real_Estate', 'Healthcare']

# Downloading the data from yahoo finance (data between start date and end date)
df = web.DataReader(indices, data_source='yahoo', start='2008-01-01', end='2015-01-01')['Close']
df.columns = newcols # changing the column name to readable ones
df = df.dropna() # Removing the null value rows to ensure concurrent time series
df = df.rolling(window=7).mean() # Rolling average (weekly) is taken
df.head(11).iloc[6:] # Viewing the first 5 non-null rows of the dataframe

,Energy,Financials,Cons_Defensive,Materials,Technology,Industrials,Utilities,Communication,Cons_Cyclical,Real_Estate,Healthcare
Date,,,,,,,,,,,
2008-01-10,77.695714,22.302426,28.390000,40.598571,25.120000,37.451429,42.869999,22.200000,31.107143,17.967143,35.557143
2008-01-11,77.138571,22.202623,28.372858,40.409999,24.857143,37.155714,43.061428,22.080000,30.830000,17.784285,35.775715
2008-01-14,76.661427,22.167808,28.360000,40.305714,24.658572,36.920000,43.270000,22.000000,30.645714,17.681428,35.924286
2008-01-15,76.148570,22.092376,28.321429,40.221428,24.531429,36.687143,43.367143,21.911428,30.524286,17.611428,36.065714
2008-01-16,75.327142,22.061043,28.221429,40.025713,24.360000,36.461429,43.228571,21.765714,30.411428,17.564286,36.121428


In [15]:
stable = '2008-09-02' # Stable value of the stock is taken as the value from Sept 02, 2008. 
event = '2008-10-01' # Date after which we expect to see the stock to go up to the stable value  

rows = []
for col in df.columns:  # Iterating for each sector
    
    df0 = df[[col]].round(2) # Defining df0 as the column of the ticker that is considered in this iteration
    ivalue = str(round(df0.loc[stable][col],1)) # Defining the initial value as the value of the stock from the stable date
    
    # Finding the date at which the stock value crosses the initial stable value calculated above.
    df1 = df0.loc[event:].reset_index() 
    proc1 = col + ' > ' + ivalue 
    index1 = df1.query(proc1).index.tolist() 
    t = df1.loc[index1[0]]['Date']
    
    # Finding the minimum stock value during the recession and the date at which the minimum value is recorded
    df2 = df0.loc[event:t].reset_index()
    minvalue = df2[col].min()
    proc2 = col + ' == ' + str(minvalue)
    index2 = df2.query(proc2).index.tolist()
    tmin = df2.loc[index2[0]]['Date']
    minvalue = round(minvalue,1)

    td = t-tmin  # difference in dates between the date when the stable value is reached after recession and the min value
    years = round(td.days/365,1) # no.of years it took to bounce back from minimum value to stable value.
    tref = str(t.month) + '/' + str(t.year) # Date at which the reference value was achieved after recession
    tmin = str(tmin.month) + '/' + str(tmin.year) # Date at which the minimum value was recorded during recession

    ivalue = float(ivalue) # Initial stable value
    percent = round((ivalue-minvalue)/minvalue*100,1) # % Gain between min value to the stable value after recession
    annpercent = round(percent/years,1) # % Annual gain 
    
    row = [col, minvalue, tmin, ivalue, tref, years, percent, annpercent]
    rows.append(row)

cols = ['Sector', 'Minimum value during drop', 'Lowest point month',  'Reference Value', 'Recovery point month', 'Years it took to bounce from min', '% Gain', '% Annual Gain']
sectordata = pd.DataFrame(rows,columns=cols)
sectordata

,Sector,Minimum value during drop,Lowest point month,Reference Value,Recovery point month,Years it took to bounce from min,% Gain,% Annual Gain
0,Energy,39.1,3/2009,74.3,2/2011,1.9,90.0,47.4
1,Financials,5.4,3/2009,16.9,10/2013,4.6,213.0,46.3
2,Cons_Defensive,19.7,3/2009,28.3,10/2010,1.6,43.7,27.3
3,Materials,18.4,3/2009,39.4,2/2011,1.9,114.1,60.1
4,Technology,13.7,3/2009,23.0,1/2010,0.8,67.9,84.9
5,Industrials,15.9,3/2009,35.1,1/2011,1.8,120.8,67.1
6,Utilities,23.2,3/2009,37.8,8/2012,3.4,62.9,18.5
7,Communication,8.2,3/2009,16.5,9/2010,1.5,101.2,67.5
8,Cons_Cyclical,16.6,3/2009,30.4,3/2010,1.0,83.1,83.1
9,Real_Estate,6.7,3/2009,18.9,4/2011,2.1,182.1,86.7
